In [21]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.insert(0,'/content/drive/MyDrive/Hybrid')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
from pso import *
from gwo import *
from bat import *
from firefly import *

In [23]:
from __future__ import division
from sklearn import svm
from sklearn.datasets import load_digits
from sklearn.model_selection import cross_val_score

import random
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation
import math

In [24]:
data = load_digits() 

In [25]:
n_samples = len(data.images)
X = data.images.reshape((n_samples, -1))
Y = data['target']

In [26]:
# modle implementation
def fitness_function1(x):  
	clf = svm.SVC(kernel='rbf', C=x[0], gamma=x[1], random_state=42)
	scores = cross_val_score(clf, X, Y, cv=5)
	
	return scores.mean()  

In [27]:
def fitness_function2(x):
	pass

def bubble_sort(fitness_values, population):
    # sort population by ascending order of fitness values
    for i in range(len(fitness_values)):
        for j in range(0, len(fitness_values)-i-1):
            if fitness_values[j] > fitness_values[j+1] :
                fitness_values[j], fitness_values[j+1] = fitness_values[j+1], fitness_values[j]
                population[j], population[j+1] = population[j+1], population[j]
    return fitness_values, population

In [28]:
def clip_pop(pop, intervals):
    # IF BOUND IS SPECIFIED THEN CLIP 'pop' VALUES SO THAT THEY ARE IN THE SPECIFIED RANGE
    return [[random.uniform(lower_bound, upper_bound) if not lower_bound <= x <= upper_bound else x for x, (lower_bound, upper_bound) in zip(sublist, intervals)] for sublist in pop]

In [29]:
class JSI:
    
    def __init__(self, **kwargs):
        self.model = kwargs.get('model', 'svm')
        self.parameters = kwargs.get('parameters', 2)
        self.intervals = kwargs.get('intervals', [[1.0, 10.0], [0.0001, 0.1]])
        self.max_iter = kwargs.get('max_iter', 10)
        self.pop_size = kwargs.get('pop_size', 10)

        if self.model == 'svm':
            self.fitness_function = fitness_function1
        elif self.model == 'decision tree':
            self.fitness_function = fitness_function2

        self.best_pop_from_all=[]
        self.best_fit_from_all=[]
        self.model_name=[]
        self.n = len(self.intervals)

        self.population = []
        if self.intervals is not None:
            for i in range(self.pop_size):
                x = [random.uniform(interval[0], interval[1]) for interval in self.intervals]
                self.population.append(x) # list of lists, for 2 dim: [[ , ], [ , ], [ , ],...,[ , ]]
            self.population = clip_pop(self.population, self.intervals)
        else:
            print('Please determine the intervals for the paremeters')
  
    def run(self):
        old_pop_bat = self.population.copy()
        old_pop_firefly = self.population.copy()
        old_pop_gwo = self.population.copy()
        old_pop_pso = self.population.copy()


        for i in range(self.max_iter):
            print('iter:',i)
            
            #global_best_position, population, fitnesses = bat_algorithm(fitness_function=self.fitness_function,pop_size=self.pop_size, intervals=self.intervals, max_iter=self.max_iter, alpha= 0.5, gamma=0.5)
            best_firefly, pop_firefly, fit_firefly = firefly_algorithm(fitness_function=self.fitness_function, pop_size=self.pop_size, dimensions=self.intervals, max_iter=self.max_iter, alpha=1.0, beta=0.5, gamma=0.1)
            best, pop_GWO, fit_pso = GWO(fitness_function=self.fitness_function, pop_size=self.pop_size,intervals=self.intervals,max_iter=self.max_iter)
            best_position, pop_pso, fit_gwo = PSO(fitness_function = self.fitness_function, pop_size=self.pop_size, intervals=self.intervals, max_iter = self.max_iter, c1 = 2, c2 = 2)


            sorted_fit_firefly, pop_firefly = bubble_sort(fit_firefly, pop_firefly)
            
            sorted_fit_gwo, pop_GWO = bubble_sort(fit_gwo, pop_GWO)
            
            sorted_fit_pso, pop_pso = bubble_sort(fit_pso, pop_pso )


            pop_firefly[self.pop_size-1] = pop_GWO[0].copy()
            pop_firefly[self.pop_size-2] = pop_pso[0].copy()

            pop_GWO[self.pop_size-1] = pop_firefly[0].copy()
            pop_GWO[self.pop_size-2] = pop_pso[0].copy()

            pop_pso[self.pop_size-1] = pop_firefly[0].copy()
            pop_pso[self.pop_size-1] = pop_GWO[0].copy

            
            old_pop_firefly = pop_firefly
            
            old_pop_gwo = pop_GWO
           
            old_pop_pso = pop_pso
        
        # self.best_pop_from_all.append(new_pop_bat[0])
        # self.best_fit_from_all.append(sorted_fit_bat[0])
        # self.model_name.append('BAT')

        self.best_pop_from_all.append(pop_firefly[0])
        self.best_fit_from_all.append(sorted_fit_firefly[0])
        self.model_name.append('Firefly')

        self.best_pop_from_all.append(pop_GWO[0])
        self.best_fit_from_all.append(sorted_fit_gwo[0])
        self.model_name.append('GWO')

        self.best_pop_from_all.append(pop_pso[0])
        self.best_fit_from_all.append(sorted_fit_pso[0])
        self.model_name.append('PSO')


        

    def best_all(self):

        best = sorted(zip(self.best_pop_from_all, self.best_fit_from_all, self.model_name), key=lambda x: x[1], reverse=True)

        return best[0][0], best[0][1], best[0][2]
    

In [30]:
model = JSI(max_iter=10, pop_size= 10)

In [32]:
model.run()

iter: 0
iter: 1
iter: 2
iter: 3
iter: 4
iter: 5
iter: 6
iter: 7
iter: 8
iter: 9


In [33]:
best_para, best_score, best_model = model.best_all()
print(f'best parameters:{best_para}, best score:{best_score}, best_model:{best_model}')

best parameters:[7.492420048027563, 0.0001], best score:0.951600123800681, best_model:PSO
